# Building LLM Applications with Gradio

## Initial Setup

In [1]:
import os, json, re, getpass, warnings
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_ollama import OllamaEmbeddings

warnings.filterwarnings('ignore')
load_dotenv(override=True)

True

In [2]:
#Check for Groq API Key
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key: ")

In [3]:
#Setting chat model
model_name = "llama-3.1-8b-instant"
llm = init_chat_model(model_name, model_provider="groq", configurable_fields=("temperature","max_tokens")) #Other Llama alternatives available are llama3-8b-8192, llama-3.3-70b-versatile

### Getting Started with Gradio `gr.Interface` 

In [4]:
import gradio as gr
def get_llm_response(input):
    output = llm.invoke(input).content
    return output

#### Basic App

Interface is Gradio's main high-level class, and allows you to create a web-based GUI / demo around a machine learning model (or any Python function) in a few lines of code. You must specify three parameters: (1) the function to create a GUI for (2) the desired input components and (3) the desired output components. Additional parameters can be used to control the appearance and behavior of the demo. See more [here](https://www.gradio.app/docs/gradio/interface). 

In [5]:
gr.close_all()
demo = gr.Interface(fn=get_llm_response, inputs="text", outputs="text")
demo.launch() #can also mention server_port as an argument (7860,7861 etc.)

* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


`demo.launch(share=True)` lets you create a public link to share with your team or friends.

#### Customizing the Labels

- Notice below that we pass in a list `[]` to `inputs` and to `outputs` because the function `fn` (in this case, `summarize()`, can take in more than one input and return more than one output.
- The number of objects passed to `inputs` list should match the number of parameters that the `fn` function takes in, and the number of objects passed to the `outputs` list should match the number of objects returned by the `fn` function.

In [6]:
gr.close_all()
demo = gr.Interface(fn=get_llm_response, 
                    inputs=[gr.Textbox(label="Input Prompt", lines=6)],
                    outputs=[gr.Textbox(label="Output by LLM", lines=3)],
                    title="My Personal Chatbot",
                    description="Chat on anything using the Llama3.1 model under the hood!"
                   )
demo.launch()

Closing server running on port: 7886
* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


### Building Gradio App using Gradio `gr.Interface` 

In [27]:
import gradio as gr
def generate(input, max_tokens):
    output = llm.with_config({"max_tokens": max_tokens}).invoke(input).content
    return output

demo = gr.Interface(fn=generate, 
                    inputs=[gr.Textbox(label="Prompt"), 
                            gr.Slider(label="Max New Tokens", 
                                      value=500,  
                                      maximum=1000, 
                                      minimum=10)], 
                    outputs=[gr.Textbox(label="Completion")])

gr.close_all()
demo.launch()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### Getting Started with Gradio `gr.Chatbot` 

Creates a chatbot that displays user-submitted messages and responses. Supports a subset of Markdown including bold, italics, code, tables. Also supports audio/video/image files, which are displayed in the Chatbot, and other kinds of files which are displayed as links. This component is usually used as an output component. See more [here](https://www.gradio.app/docs/gradio/chatbot).

- `gr.Chatbot()` allows you to save the chat history (between the user and the LLM) as well as display the dialogue in the app.
- Define your `fn` to take in a `gr.Chatbot()` object.  
  - Within your defined `fn` function, extend with a list containing the user message and the LLM's response in OpenAI's messages format that's widely used:
`chatbot_object.append( [{'role':'user','content':user_message}, {'role':'assistant','content':llm_message}] )`

- Include the chatbot object in both the inputs and the outputs of the app.

#### Basic App

In [39]:
import random

def respond(message, chat_history):
        #No LLM here, just respond with a random pre-made message
        bot_message = random.choice(["Tell me more about it", 
                                     "Cool, but I'm not interested", 
                                     "Hmmmm, ok then"]) 
        chat_history.extend([{'role':'user','content': message},
                            {'role':'assistant','content': bot_message}])
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240, type='messages') #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


#### Format the prompt with the chat history

- You can iterate through the chatbot object with a for loop.
- Each item is a tuple containing the user message and the LLM's message.

```Python
for turn in chat_history:
    user_msg, bot_msg = turn
    ...
```

In [52]:
def format_chat_prompt(message, chat_history):
    prompt = ""
    for i in range(len(chat_history) - 1):
    # Ensure that we are processing pairs and not going out of bounds
        if i % 2 == 0: # Only process if i is an even number (start of a pair)
            user_message = chat_history[i]["content"]
            bot_message = chat_history[i+1]["content"]
            prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

In [53]:
def respond(message, chat_history):
        formatted_prompt = format_chat_prompt(message, chat_history)
        bot_message = llm.with_config({"max_tokens": 1024}).invoke(formatted_prompt).content
        chat_history.extend([{'role':'user','content': message},
                            {'role':'assistant','content': bot_message}])
        return "", chat_history

In [54]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240, type='messages') #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

In [55]:
gr.close_all()
demo.launch()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


#### Advanced Feature - System Prompt

In [59]:
def format_chat_prompt(message, chat_history, instruction):
    prompt = f"System:{instruction}"
    for i in range(len(chat_history) - 1):
    # Ensure that we are processing pairs and not going out of bounds
        if i % 2 == 0: # Only process if i is an even number (start of a pair)
            user_message = chat_history[i]["content"]
            bot_message = chat_history[i+1]["content"]
            prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

In [60]:
def respond(message, chat_history):
        instruction = "You are a rockstar! Respond accordingly."
        formatted_prompt = format_chat_prompt(message, chat_history, instruction)
        bot_message = llm.with_config({"max_tokens": 1024}).invoke(formatted_prompt).content
        chat_history.extend([{'role':'user','content': message},
                            {'role':'assistant','content': bot_message}])
        return "", chat_history

In [61]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240, type='messages') #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

In [62]:
#Other functions remain the same!
gr.close_all()
demo.launch()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.
